#### Descomposición LU para resolver sistemas lineales

Es usual resolver el sistema $A\textbf{x}=\textbf{b}$ aplicando operaciones elementales de fila sobre $A$ para construir un sistema reducido equivalente $U\textbf{x}=\textbf{c}$. Las operaciones permitidas sobre las filas de $A$ son de tres tipos:

1) Multiplicar una fila por un escalar no nulo

2) Intercambiar dos filas

3) Sumas un múltiplo escalar de una fila con otra fila

Imagine que debe resolver el sistema 
$$x + y + z = 1$$ 
$$x + 4y + 2z = 3$$
$$4x + 7y + 8z = 9$$

El sistema se escribe en su forma matricial de la siguiente forma $A \textbf{x} = \textbf{b}$ con 
$A = \begin{bmatrix}
    1 & 1 & 1 \\
    1 & 4 & 2 \\
    4 & 7 & 8 \\
\end{bmatrix}$ y 
$\textbf{b } = \begin{bmatrix}
   1 \\
   3 \\
   9 \\
\end{bmatrix}$

**Exploración de operaciones elementales de fila en NumPy**

In [1]:
# La forma de importar NumPy y de inicializar una matriz es:

import numpy as np

A = np.array([[1,1,1,1],
              [1,4,2,3],
              [4,7,8,9]], dtype=np.float64)

In [2]:
# Poner ceros bajo el pivote de la primera fila
A[1,0:] -= (A[1,0] / A[0,0]) * A[0]
A[2,0:] -= (A[2,0] / A[0,0]) * A[0]
A

array([[1., 1., 1., 1.],
       [0., 3., 1., 2.],
       [0., 3., 4., 5.]])

In [3]:
# Poner ceros bajo el pivote de la segunda fila
A[2,1:] -= (A[2,1] / A[1,1]) * A[1,1:]
A

array([[1., 1., 1., 1.],
       [0., 3., 1., 2.],
       [0., 0., 3., 3.]])

##### **Ejercicio 1**

Escriba una función en Python que reciba una matriz aumentada $[A|b]$ (asuma que al escalonar $A$ no se requiren intercambios de filas) y luego retorne una matriz $U$ y $c$, asociados al sistema $[U|c]$ equivalente por filas al sistema original. (**Sug**: En las condiciones dadas, use operaciones tipo 3)).

Ejemplifique con matrices chicas que su función está trabajando bien. Considdere `np.random.randint()` si necesita generar casos para verificar.

In [4]:
def escalonar(A_b):
    """entrada: A_b ndarray. concatenación de A y b, del sistema Ax=b.
        salida: U, c. U es escalonada. Ux=c es el sistema reducido."""
    m, n = A.shape
    U_c = A_b.copy()
    for j in range(n):
        for i in range(j+1, m):
            U_c[i,j:] -= (U_c[i,j] / U_c[j,j]) * U_c[j,j:]
    U, c = U_c[:, :-1], U_c[:, -1]
    return U, c

In [5]:
A = np.array([[1,1,1,1],
              [1,4,2,3],
              [4,7,8,9]], dtype=np.float64)
escalonar(A)


(array([[1., 1., 1.],
        [0., 3., 1.],
        [0., 0., 3.]]),
 array([1., 2., 3.]))

Finalizado el proceso de construcción del sistema reducido equivalente $U\textbf{x} = \textbf{c}$, el problema se puede resolver por sustitución regresiva 

In [6]:
def matriz_de_coef_aleat(n_rows):
    """entrada: n_rows int que determina número de filas y columnas de A
        salida: A_b matriz aumentada de un sistema lineal con valores aleatorios"""
    A_b = np.random.randint(-5,5, (n_rows, n_rows+1)) / 1.0 # se divide por 1.0 para tener flaots
    return A_b

In [7]:
A_b = matriz_de_coef_aleat(4)
escalonar(A_b)[0].round(2)

array([[-4.  ,  1.  ,  4.  , -1.  ],
       [ 0.  , -4.75, -4.  , -3.25],
       [ 0.  ,  0.  , -6.53,  0.95],
       [-4.  ,  1.  , -1.  ,  4.  ]])

#### Sobre descomposición LU

Es posible factorizar $A$ en la forma $LU$ donde $L$ es producto de matrices elementales tipo III y $U$ es triangular superior (escalonada equivalente por filas a $A$, sin uso de intercambio de filas). En dicho caso:

$$E _{k}  \cdots E _{ 2} E _{ 1} A = U \quad \to \quad  A=(E _{k}  \cdots E _{ 2} E _{ 1}) ^{-1}U  \quad \to \quad A=E _{1} ^{-1}  E _{ 2} ^{-1}  \cdots E _{ k} ^{-1}U \quad \to \quad  A=LU$$ 

##### **Ejercicio 2**

Defina una función que practique la descomposición $LU$ asumiendo que no se requieren intercambios de filas.


Una forma de implementar la descomposición $LU$: 

In [8]:
def LUdecomposition(A):
    m, n = A.shape
    L, U = np.eye(m), A.copy()
    for j in range(n):
        for i in range(j+1, m):
            L[i,j] = U[i,j] / U[j,j]
            U[i,j:] -= L[i,j] * U[j,j:]
    return L, U


In [9]:
A = np.arange(9).reshape((3,3)) / 1.0 + np.eye(3)
print("A: \n", A)
L, U = LUdecomposition(A)
print("L:\n", L)
print("U:\n", U)


A: 
 [[1. 1. 2.]
 [3. 5. 5.]
 [6. 7. 9.]]
L:
 [[1.  0.  0. ]
 [3.  1.  0. ]
 [6.  0.5 1. ]]
U:
 [[ 1.   1.   2. ]
 [ 0.   2.  -1. ]
 [ 0.   0.  -2.5]]


In [10]:
# El algoritmo anterior se puede mejorar

def fastLUdecomposition(A): # revisar el problema con esta funcion
    m, n = A.shape
    L, U = np.eye(m), A.copy()
    for k in range(n-1):
        L[(k+1):, k] = U[(k+1):, k] / U[k,k]
        U[(k+1):, k:] -= np.dot(L[(k+1):, k].reshape(-1, 1), U[k, k:].reshape(1, -1))
    return L, U

#### Sobre $LU$ para resolver sistemas


Si $A = LU$ y $A\textbf{x}=\textbf{b}$, entonces $LU\textbf{x}=A\textbf{x} =\textbf{b}$. Resolver los sistemas triangulares: 

**1)** $L\textbf{y}=\textbf{b}$ 

**2)** $U\textbf{x}=\textbf{y}$


De $L\textbf{y}=\textbf{b}$ con $\textbf{y}=[y _{ 1} ,\cdots , y _{ n} ]$ y $\textbf{b}= [b _{ 1} ,\cdots , b _{ n} ]$, se deduce que
$$y _{k} = b _{k} - \displaystyle \sum _{j=1} ^{ k-1} l _{k j}y _{ j}      $$


De $U\textbf{x}=\textbf{y}$ con $\textbf{y}=[y _{ 1} ,\cdots , y _{ n} ]$ y $\textbf{x}= [x _{ 1} ,\cdots , x _{ n} ]$, se deduce que
$$x _{k} = \frac{1}{u _{k k} } \left ( y _{k} - \displaystyle \sum _{ j=k+1} ^{ n} u _{k j}x _{ j}    \right ) $$   


##### **Ejercicio 3**

Crear una función que reciba $A$ y $\textbf{b}$ asociados al sistema $A \textbf{x} = \textbf{b}$ y utilizando las sustituciones previas retorne la solucón $\textbf{x}$ 

In [11]:
def solve_Ly_Pb(L, P, b): # Use P = I, bajo hipótesis de NO intercambios de filas
    n = len(b)
    ys = []
    for k in range(1, n+1):
        s = np.array([L[k-1,j-1] * ys[j-1] for j in range(1, k)]).sum()
        yk = b[k-1] - s
        ys.append(yk)
    return np.array(ys)

def solve_Ux_y(U, y):
    n = len(y)
    xs = [(1/U[-1,-1]) * y[-1]] # primero, la ultima incognita
    for k in range(n-2, -1, -1):
        s = np.dot(np.array(xs), U[k, :][-len(xs)])[0]
        xk = (1/U[k, k]) * (y[k] - s)
        xs.insert(0, xk)
    return np.array(xs)


def solve_linear_system(A, b):
    """ suponiendo que A factoriza LU, sin intercambio de filas"""
    L, U = fastLUdecomposition(A)
    n = A.shape[0]
    P = np.eye(n)
    y = solve_Ly_Pb(L, P, b)
    return solve_Ux_y(U, y)


In [12]:
# Verifiquemos con un ejemplo

S = np.array([[2,-1,0,0],
              [-1,2,-1,0],
              [0, -1,2,-1],
              [0,0,-1,2]], dtype=np.float64)
              
L1, U1 = fastLUdecomposition(S)
print(L1)
print('')
print(U1)
print('')
print(L1 @ U1)

# a continuacion un ejemplo de solucion de Sx=e1, donde x debe ser la primero col de S.inv()

y = solve_Ly_Pb(L1, np.eye(len(L1)), np.eye(len(L1))[0])
x = solve_Ux_y(U1, y)
print(x)
print('por otro lado')
print(np.linalg.inv(S)[:, 0])

[[ 1.          0.          0.          0.        ]
 [-0.5         1.          0.          0.        ]
 [ 0.         -0.66666667  1.          0.        ]
 [ 0.          0.         -0.75        1.        ]]

[[ 2.         -1.          0.          0.        ]
 [ 0.          1.5        -1.          0.        ]
 [ 0.          0.          1.33333333 -1.        ]
 [ 0.          0.          0.          1.25      ]]

[[ 2. -1.  0.  0.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [ 0.  0. -1.  2.]]
[0.8 0.6 0.4 0.2]
por otro lado
[0.8 0.6 0.4 0.2]


#### Aplicaciones de la descomposición $LU$ 

La descomposición $LU$ puede ser usada para calcular inversas y determinantes con relativa eficiencia.

**Inversa**

 $A = LU \quad \Rightarrow \quad LU A ^{-1} = I $. Al resolver los sistemas con sustitución hacia atrás y hacia adelante: $LU a _{ i} = e _{ i} $ (donde $e _{ i} $ son columnas de la identidad), entonces las soluciones $a _{ i} $ ubicadas como columnas, constituirán:
$$A ^{-1}  = \begin{bmatrix} 
    a _{ 1} & a _{ 2} & \cdots & a _{ n} \\
    | &| & \  &| \\
 \end{bmatrix}$$ 


 **Determinante**

 $\det (A) = \det(L) \det(U) = \displaystyle \prod _{i} ^{ n} l _{ ii} \prod _{i} ^{ n} u _{ ii} = \displaystyle 1\prod _{i} ^{ n} u _{ ii}  $ como consecuencia de que $L$ y $U$ son triangulares.

#### Aplicación a ecuación diferencial bajo condiciones de fontera
$$\displaystyle - \frac{d ^{ 2} u}{dx ^{ 2} }= f(x), \quad \quad 0\leq x\leq 1, \quad u(x) = 0, \quad u(1) = 0 $$

Tomar comoreferencia Strang page 76